In [1]:
"""
Regras de risco em reembolsos
-----------------------------
Este script lê um CSV com dados de reembolsos, cria **apenas** as colunas-bandeira
(`True` / `False`) correspondentes às nove regras de fraude definidas pelo especialista
e devolve um DataFrame pronto para análise posterior.

📌 Colunas adicionadas
    regra_1_valor_contrato
    regra_2_valor_maior_3000
    regra_3_tres_maiores_500
    regra_4_dois_maiores_1000
    regra_5_total_maior_70p_contrato
    regra_6_aprovacao_rapida
    regra_7_horario_tarde
    regra_8_variacao_ambas_maior_30
    regra_9_variacao_valor_dias_agenciado_maior_30
"""

import pandas as pd


def adicionar_regras_reembolso(arquivo_csv: str) -> pd.DataFrame:
    # ────────────────────────────────
    # 1) Carregamento e pré-processamento mínimo
    # ────────────────────────────────
    df = pd.read_csv(arquivo_csv)

    date_cols = [
        "Data da criação do reembolso",
        "Data da ultima atualização do reembolso",
        "Data da aprovação ou rejeição do reembolso",
    ]
    df[date_cols] = df[date_cols].apply(
        pd.to_datetime, errors="coerce", utc=False  # assume fuso local já embutido
    )

    # ────────────────────────────────
    # 2) Regras INDIVIDUAIS (por transação)
    # ────────────────────────────────
    df["regra_1_valor_contrato"] = (
        # (df["Valor do contrato"] >= 500)
        # & 
        (df["Valor total do reembolso"] >= 0.5 * df["Valor do contrato"])
    )

    df["regra_2_valor_maior_3000"] = df["Valor total do reembolso"] > 3000

    df["regra_6_aprovacao_rapida"] = (
        df["Data da aprovação ou rejeição do reembolso"].notna()
        & df["Data da criação do reembolso"].notna()
        & (
            (
                df["Data da aprovação ou rejeição do reembolso"]
                - df["Data da criação do reembolso"]
            ).dt.total_seconds()
            / 60
            < 15
        )
        #& (df["Valor total do reembolso"] > 500)
    )

    df["regra_7_horario_tarde"] = (
        df["Data da criação do reembolso"].dt.hour >= 21
    ) & (df["Valor total do reembolso"] > 300)

    # ────────────────────────────────
    # 3) Regras AGREGADAS (por dia | transportadora | motorista)
    # ────────────────────────────────
    df["Data da criação do reembolso Date"] = df[
        "Data da criação do reembolso"
    ].dt.date

    agrupado = (
        df.groupby(
            [
                "Data da criação do reembolso Date",
                "Nome da transportadora",
                "Nome do motorista",
                "Valor do contrato",
                "Número de dias agenciados do contrato",
            ],
            as_index=False,
        )
        .agg(
            total_reembolsos=("Valor total do reembolso", "sum"),
            num_reembolsos=("Valor total do reembolso", "count"),
            num_reembolsos_alto_valor=(
                "Valor total do reembolso",
                lambda x: (x > 500).sum(),
            ),
            num_reembolsos_valor_muito_alto=(
                "Valor total do reembolso",
                lambda x: (x > 1000).sum(),
            ),
        )
        .sort_values(
            ["Nome da transportadora", "Nome do motorista", "Data da criação do reembolso Date"]
        )
    )

    # Regras 3, 4, 5
    agrupado["regra_3_tres_maiores_500"] = agrupado[
        "num_reembolsos_alto_valor"
    ] >= 3
    agrupado["regra_4_dois_maiores_1000"] = agrupado[
        "num_reembolsos_valor_muito_alto"
    ] >= 2
    agrupado["regra_5_total_maior_70p_contrato"] = (
        agrupado["total_reembolsos"] > 0.7 * agrupado["Valor do contrato"]
    )

    # Variações p/ regras 8 e 9
    agrupado["variacao_num_reembolsos"] = (
        agrupado.groupby(["Nome da transportadora", "Nome do motorista"])["num_reembolsos"]
        .pct_change()
        .abs()
    )
    agrupado["variacao_total_reembolsos"] = (
        agrupado.groupby(["Nome da transportadora", "Nome do motorista"])["total_reembolsos"]
        .pct_change()
        .abs()
    )

    agrupado.dropna(
        subset=["variacao_num_reembolsos", "variacao_total_reembolsos"], inplace=True
    )

    agrupado["regra_8_variacao_ambas_maior_30"] = (
        (agrupado["variacao_num_reembolsos"] >= 0.30)
        & (agrupado["variacao_total_reembolsos"] >= 0.30)
    )

    agrupado["regra_9_variacao_valor_dias_agenciado_maior_30"] = (
        (agrupado["variacao_total_reembolsos"] / agrupado["Número de dias agenciados do contrato"])
        .abs()
        >= 0.30
    )

    # ────────────────────────────────
    # 4) Junta regras agregadas de volta ao DataFrame principal
    # ────────────────────────────────
    df = df.merge(
        agrupado[
            [
                "Data da criação do reembolso Date",
                "Nome da transportadora",
                "Nome do motorista",
                "regra_3_tres_maiores_500",
                "regra_4_dois_maiores_1000",
                "regra_5_total_maior_70p_contrato",
                "regra_8_variacao_ambas_maior_30",
                "regra_9_variacao_valor_dias_agenciado_maior_30",
            ]
        ],
        on=[
            "Data da criação do reembolso Date",
            "Nome da transportadora",
            "Nome do motorista",
        ],
        how="left",
    )

    # Qualquer NaN virou False
    regra_cols = [c for c in df.columns if c.startswith("regra_")]
    df[regra_cols] = df[regra_cols].fillna(False)

    # Remove coluna auxiliar de data
    df.drop(columns="Data da criação do reembolso Date", inplace=True)

    return df


# ────────────────────────────────
# Uso básico
# ────────────────────────────────
if __name__ == "__main__":
    caminho_csv = "src/data/reembolsos.csv"
    df_com_regras = adicionar_regras_reembolso(caminho_csv)
    df_com_regras.to_csv("src/data/output/reembolsos_com_regras.csv", index=False)
    print("Arquivo salvo com colunas de regras adicionadas.")


/tmp/ipykernel_21212/1949497073.py:166: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[regra_cols] = df[regra_cols].fillna(False)


Arquivo salvo com colunas de regras adicionadas.


In [2]:
df = adicionar_regras_reembolso("src/data/reembolsos.csv")
df.head()

/tmp/ipykernel_21212/1949497073.py:166: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[regra_cols] = df[regra_cols].fillna(False)


,ID do Reembolso,Data da criação do reembolso,Data da ultima atualização do reembolso,Data da aprovação ou rejeição do reembolso,Status do reembolso,ID do contrato,Valor total do reembolso,Motivo do reembolso,Descrição do motivo do reembolso,Tipo do reembolso,...,Nome da transportadora,regra_1_valor_contrato,regra_2_valor_maior_3000,regra_6_aprovacao_rapida,regra_7_horario_tarde,regra_3_tres_maiores_500,regra_4_dois_maiores_1000,regra_5_total_maior_70p_contrato,regra_8_variacao_ambas_maior_30,regra_9_variacao_valor_dias_agenciado_maior_30
0,66220,2024-12-30 15:34:35,2024-12-30 16:54:11,2024-12-30 16:54:11,Reprovado,130463,92,6.0,Comprovante errado,1,...,GT FOODS - BARUERI,False,False,False,False,False,False,False,False,False
1,31526,2024-01-18 15:44:21,2024-01-18 15:51:51,2024-01-18 15:51:51,Reprovado,59215,65,0.0,NaN,101,...,GT FOODS - BARUERI,False,False,True,False,False,False,False,False,False
2,31603,2024-01-19 10:46:09,2024-01-19 10:58:16,2024-01-19 10:58:16,Reprovado,59485,270,0.0,NaN,101,...,GT FOODS - BARUERI,False,False,True,False,False,False,False,False,False
3,48047,2024-07-22 16:01:36,2024-07-22 16:08:59,2024-07-22 16:08:59,Reprovado,93563,32,1.0,Duplicidade,1,...,TRANSPORTES TOZZO LTDA,False,False,True,False,False,False,False,True,False
4,30580,2024-01-09 16:56:56,2024-01-10 12:25:05,2024-01-10 12:24:48,Pago,58564,70,NaN,NaN,101,...,PETRÓPOLIS - FÁBRICA UBERABA,False,False,False,False,False,False,False,False,False


In [3]:
columns = [ 'regra_1_valor_contrato', 'regra_2_valor_maior_3000',
       'regra_6_aprovacao_rapida', 'regra_7_horario_tarde',
       'regra_3_tres_maiores_500', 'regra_4_dois_maiores_1000',
       'regra_5_total_maior_70p_contrato', 'regra_8_variacao_ambas_maior_30',
       'regra_9_variacao_valor_dias_agenciado_maior_30']
df[columns].sum()

regra_1_valor_contrato                             1970
regra_2_valor_maior_3000                             65
regra_6_aprovacao_rapida                          10568
regra_7_horario_tarde                               234
regra_3_tres_maiores_500                            227
regra_4_dois_maiores_1000                           261
regra_5_total_maior_70p_contrato                   3010
regra_8_variacao_ambas_maior_30                   17992
regra_9_variacao_valor_dias_agenciado_maior_30     6214
dtype: int64

In [4]:
df["Risco"] = df[columns].mean(axis=1)

In [7]:
df.sort_values("Risco", ascending=False).head(5)

,ID do Reembolso,Data da criação do reembolso,Data da ultima atualização do reembolso,Data da aprovação ou rejeição do reembolso,Status do reembolso,ID do contrato,Valor total do reembolso,Motivo do reembolso,Descrição do motivo do reembolso,Tipo do reembolso,...,regra_1_valor_contrato,regra_2_valor_maior_3000,regra_6_aprovacao_rapida,regra_7_horario_tarde,regra_3_tres_maiores_500,regra_4_dois_maiores_1000,regra_5_total_maior_70p_contrato,regra_8_variacao_ambas_maior_30,regra_9_variacao_valor_dias_agenciado_maior_30,Risco
28147,53846,2024-09-10 21:19:55,2024-09-11 13:44:49,2024-09-10 21:27:48,Pago,107702,3200,NaN,NaN,0,...,True,True,True,True,True,True,True,True,True,1.000000
31087,56710,2024-10-05 23:28:46,2024-10-05 23:35:07,2024-10-05 23:33:56,Pago,113767,2468,NaN,NaN,2,...,True,False,True,True,True,True,True,True,True,0.888889
27687,53402,2024-09-06 19:25:29,2024-09-07 10:03:20,2024-09-06 19:27:16,Pago,106937,3600,NaN,NaN,0,...,True,True,True,False,True,True,True,True,True,0.888889
27254,53001,2024-09-03 22:38:11,2024-09-03 22:45:06,2024-09-03 22:44:08,Pago,105964,890,NaN,NaN,0,...,True,False,True,True,True,True,True,True,True,0.888889
27258,52999,2024-09-03 22:19:57,2024-09-03 22:25:08,2024-09-03 22:23:03,Pago,105964,2850,NaN,NaN,4,...,True,False,True,True,True,True,True,True,True,0.888889
